In [2]:
## Import libraries from Python(gesture)
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import cv2


In [3]:
##Load Dataset (leapGestRecog)
DATASET_DIR = "leapGestRecog"

IMG_SIZE = 64  # small size = optimized model
X = []
y = []

label_map = {}
label_index = 0

for root, dirs, files in os.walk(DATASET_DIR):
    for file in files:
        if file.endswith(".png"):
            path = os.path.join(root, file)
            label = root.split("/")[-1]

            if label not in label_map:
                label_map[label] = label_index
                label_index += 1

            img = cv2.imread(path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            X.append(img)
            y.append(label_map[label])

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = to_categorical(y, num_classes=len(label_map))

print("Dataset Loaded Successfully")
print("Total Images:", len(X))
print("Classes:", label_map)


Dataset Loaded Successfully
Total Images: 20000
Classes: {'leapGestRecog\\00\\01_palm': 0, 'leapGestRecog\\00\\02_l': 1, 'leapGestRecog\\00\\03_fist': 2, 'leapGestRecog\\00\\04_fist_moved': 3, 'leapGestRecog\\00\\05_thumb': 4, 'leapGestRecog\\00\\06_index': 5, 'leapGestRecog\\00\\07_ok': 6, 'leapGestRecog\\00\\08_palm_moved': 7, 'leapGestRecog\\00\\09_c': 8, 'leapGestRecog\\00\\10_down': 9, 'leapGestRecog\\01\\01_palm': 10, 'leapGestRecog\\01\\02_l': 11, 'leapGestRecog\\01\\03_fist': 12, 'leapGestRecog\\01\\04_fist_moved': 13, 'leapGestRecog\\01\\05_thumb': 14, 'leapGestRecog\\01\\06_index': 15, 'leapGestRecog\\01\\07_ok': 16, 'leapGestRecog\\01\\08_palm_moved': 17, 'leapGestRecog\\01\\09_c': 18, 'leapGestRecog\\01\\10_down': 19, 'leapGestRecog\\02\\01_palm': 20, 'leapGestRecog\\02\\02_l': 21, 'leapGestRecog\\02\\03_fist': 22, 'leapGestRecog\\02\\04_fist_moved': 23, 'leapGestRecog\\02\\05_thumb': 24, 'leapGestRecog\\02\\06_index': 25, 'leapGestRecog\\02\\07_ok': 26, 'leapGestRecog\\02\

In [4]:
##Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)


In [5]:
##Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

datagen.fit(X_train)


In [6]:
##Build CNN Model (convolutional Neural Network)
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(len(label_map), activation='softmax')
])

model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(0.0005),
    metrics=["accuracy"]
)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 62, 62, 32)       128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 29, 29, 64)       256       
 hNormalization)                                                 
                                                        

In [7]:
##Train Model
checkpoint = ModelCheckpoint("gesture_model.h5", save_best_only=True, monitor="val_accuracy")
early_stop = EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_test, y_test),
    epochs=20,
    callbacks=[checkpoint, early_stop]
)


Epoch 1/20
500/500 [==============================] - 34s 66ms/step - loss: 3.7367 - accuracy: 0.1433 - val_loss: 1.5024 - val_accuracy: 0.5972
Epoch 2/20
500/500 [==============================] - 32s 64ms/step - loss: 1.6337 - accuracy: 0.5290 - val_loss: 0.2796 - val_accuracy: 0.9147
Epoch 3/20
500/500 [==============================] - 33s 65ms/step - loss: 0.7611 - accuracy: 0.7617 - val_loss: 0.1107 - val_accuracy: 0.9628
Epoch 4/20
500/500 [==============================] - 33s 66ms/step - loss: 0.4577 - accuracy: 0.8516 - val_loss: 0.0546 - val_accuracy: 0.9805
Epoch 5/20
500/500 [==============================] - 34s 68ms/step - loss: 0.2876 - accuracy: 0.9016 - val_loss: 0.0596 - val_accuracy: 0.9758
Epoch 6/20
500/500 [==============================] - 33s 66ms/step - loss: 0.2331 - accuracy: 0.9211 - val_loss: 0.0563 - val_accuracy: 0.9772
Epoch 7/20
500/500 [==============================] - 33s 67ms/step - loss: 0.1955 - accuracy: 0.9327 - val_loss: 0.0608 - val_accuracy:

In [8]:
##Evaluate Model
loss, acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", acc)


125/125 [==============================] - 2s 20ms/step - loss: 0.0150 - accuracy: 0.9898
Test Accuracy: 0.9897500276565552


In [9]:
##Save Model + Label Map
model.save("gesture_model.h5")

with open("label_map.json", "w") as f:
    json.dump(label_map, f)

print("Model & Labels Saved Successfully")


Model & Labels Saved Successfully


In [10]:
##MODEL SIZE OPTIMIZATION
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("gesture_model.tflite", "wb") as f:
    f.write(tflite_model)

print("Optimized model saved")


INFO:tensorflow:Assets written to: C:\Users\maury\AppData\Local\Temp\tmpox67_9iq\assets


INFO:tensorflow:Assets written to: C:\Users\maury\AppData\Local\Temp\tmpox67_9iq\assets


Optimized model saved
